In [ ]:
#                 Summit at Snoqualmie : 47.409478, -121.421320
#     Crystal Mtn Resort (chair 6 spot): 46.922385, -121.502320
# Mtn Baker Ski Area (Heather Meadows) : 48.863, -121.677
# Stevens Pass (Top of Juputer Express): 47.731791, -121.085012
#           Alpental (Top of ArmStrong): 47.435697, -121.434410

In [ ]:
 # Build a forecast dictionary.
        # forecast = {
        #     "latitude" : latitude,
        #     "longitude": longitude,
        #     "forecastdata_periods" : forecastdata.get("properties", {}).get("periods", []),
        #     "elevation" : forecastdata.get("properties", {}).get("elevation", {"unitCode": "", "value": 0}),
        #     "location" : points.get("properties", {}).get("relativeLocation", {}).get("properties", {}),
        #     "detailedForecastPlot" : img_base64,
        # }

In [ ]:
# Append the forecast dictionary to the session-stored list
        # Check if there's already a list of forecasts stored in the session. If not, create an empty list.
        # Update the session with the latest forecast after appending the new forecast.
        # forecasts: list = Session.get("forecasts", [])
        # forecasts.append(forecast)
        # Session["forecasts"] = forecasts

        # Redirect to GET to display the forecast[s].
    #     return redirect(url_for("index"))
    
    # forecasts = Session.get("forecasts", [])